In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/train.csv


In [2]:
train_data=pd.read_csv('/kaggle/input/titanic/train.csv')
train_data.describe()
train_data.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [3]:
train_data.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [4]:
train_data.shape

(891, 12)

In [5]:
#Data processing for train data

missing_val_count_by_column = (train_data.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

Age         177
Cabin       687
Embarked      2
dtype: int64


In [6]:
#dropping the column Cabin as it is more than 70 % missing in both test and train
train_data=train_data.drop(["Cabin"],axis=1)


In [7]:
train_data.shape

(891, 11)

In [8]:
#dealing with Age and embarked columns for missing data
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values = np.nan, strategy = 'most_frequent')

imputer=imputer.fit(train_data[['Embarked']])
train_data[['Embarked']]=imputer.transform(train_data[['Embarked']])

In [9]:
#dealing with Age and embarked columns for missing data
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values = np.nan, strategy = 'mean')

imputer=imputer.fit(train_data[['Age']])
train_data[['Age']]=imputer.transform(train_data[['Age']])

In [10]:
#as name and ticket are going to different for everyone there is no benefit in keeping them
train_data=train_data.drop(['Name','Ticket'],axis=1)

In [11]:
#filtering the categorical columns from the data set
categorical_feature_mask = train_data.dtypes==object

In [12]:
#transfering categorical columns into a list
categorical_cols = train_data.columns[categorical_feature_mask].tolist()
train_data[categorical_cols].head(10)

,Sex,Embarked
0,male,S
1,female,C
2,female,S
3,female,S
4,male,S
5,male,Q
6,male,S
7,male,S
8,female,S
9,female,C


In [13]:
#Encoding the categorical data
from sklearn.preprocessing import LabelEncoder
labelencoder_train=LabelEncoder()
train_data[categorical_cols] = train_data[categorical_cols].apply(lambda col: labelencoder_train.fit_transform(col))

In [14]:
train_data.head(10)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,1,22.000000,1,0,7.2500,2
1,2,1,1,0,38.000000,1,0,71.2833,0
2,3,1,3,0,26.000000,0,0,7.9250,2
3,4,1,1,0,35.000000,1,0,53.1000,2
4,5,0,3,1,35.000000,0,0,8.0500,2
5,6,0,3,1,29.699118,0,0,8.4583,1
6,7,0,1,1,54.000000,0,0,51.8625,2
7,8,0,3,1,2.000000,3,1,21.0750,2
8,9,1,3,0,27.000000,0,2,11.1333,2
9,10,1,2,0,14.000000,1,0,30.0708,0


In [15]:
X=train_data.iloc[:,[0,2,3,4,5,6,7,8]]
y=train_data.iloc[:,1]

In [16]:
# Fitting Random Forest Regression to the dataset
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)
regressor.fit(X,y)


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

In [17]:
test_data=pd.read_csv('/kaggle/input/titanic/test.csv')
test_data.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [18]:
test_data.columns

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [19]:
test_data.shape

(418, 11)

In [20]:
missing_val_count_by_column = (test_data.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

Age       86
Fare       1
Cabin    327
dtype: int64


In [21]:
#dropping the column Cabin as it is more than 70 % missing in both test and train
test_data=test_data.drop(["Cabin"],axis=1)


In [22]:
#dealing with Age and embarked columns for missing data
from sklearn.impute import SimpleImputer
imputer_t = SimpleImputer(missing_values = np.nan, strategy = 'mean')

imputer_t=imputer_t.fit(test_data[['Age','Fare']])
test_data[['Age','Fare']]=imputer_t.transform(test_data[['Age','Fare']])

In [23]:
#as name and ticket are going to different for everyone there is no benefit in keeping them
test_data=test_data.drop(['Name','Ticket'],axis=1)

In [24]:
#filtering the categorical columns from the data set
categorical_feature_mask = test_data.dtypes==object

In [25]:
#transfering categorical columns into a list
categorical_cols = test_data.columns[categorical_feature_mask].tolist()
test_data[categorical_cols].head(10)

,Sex,Embarked
0,male,Q
1,female,S
2,male,Q
3,male,S
4,female,S
5,male,S
6,female,Q
7,male,S
8,female,C
9,male,S


In [26]:
#Encoding the categorical data
from sklearn.preprocessing import LabelEncoder
labelencoder_test=LabelEncoder()
test_data[categorical_cols] = test_data[categorical_cols].apply(lambda col: labelencoder_test.fit_transform(col))

In [27]:
#predicting new result

train_predictions=regressor.predict(X)

In [28]:
test_predictions=regressor.predict(test_data)

In [29]:
output=pd.DataFrame({'PassengerId':test_data.PassengerId,'Survived':test_predictions})

In [30]:
output.to_csv('submission_1.csv',index=False)